# Axis of interest

In [1]:
import os
import json
import lmstudio as lms
import sys
import re
from langchain_openai import ChatOpenAI
from pathlib import Path

repo_root = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path("/Users/jpconde/Facultad/tesis")
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

In [8]:
from axis_of_interest.schemas import list_of_aoi, indice_aoi
from axis_of_interest.prompts import prompt_generate_plot_schema, prompt_generate_cuento, prompt_generar_axis_of_interest
from axis_of_interest.utils import render_aoi_md, render_plot_schema_md

### Crear AoI en python

Pasamos los AoI del Dr.Gervás a python. Usamos pydantic


In [3]:
for aoi in list_of_aoi:
    print(render_aoi_md(aoi))
    print("\n")


AXISofINTEREST =  CALL TO ACTION REWARD
PROTAGONIST =  called
ROLES =  called caller rewarded
────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Call

  CallToAction  characters(called=Hero,caller=Sender)

────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Reward

  Reward  characters(rewarded=Hero)

────────────────────────────────────────────────────────────────



AXISofINTEREST =  CONFLICT
PROTAGONIST =  attacker
ROLES =  attacker defender winner loser
────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Struggle

  Struggle  characters(attacker=X,defender=Y)

────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Victory

  Victory  characters(winner=X,loser=Y)

────────────────────────────────────────────────────────────────



AXISofINTEREST =  DONOR
PROTAGONIST =  tested
ROLES =  tested tester user gift
───────────────────────────────────────────────────────

### Crear Plot Schema

Le pegamos a una api de un LLM, con algunos axes of interest en el prompt y le pedimos que nos genere un Plot Schema

In [4]:
print(indice_aoi)

{'CALL TO ACTION REWARD': 0, 'CONFLICT': 1, 'DONOR': 2, 'INTERDICTION VIOLATED': 3, 'JOURNEY': 4, 'KIDNAPPING': 5, 'KIDNAPPING PUNISHMENT': 6, 'PURSUIT': 7, 'RAGS 2 RICHES': 8, 'RELENTING GUARDIAN': 9, 'REPENTANCE': 10, 'RIVALRY': 11, 'SHIFTING LOVE': 12, 'TASK': 13, 'UNRELENTING GUARDIAN': 14}


In [5]:
model1 = ChatOpenAI(
    model="gpt-5",
    temperature=0.0,
    #model_kwargs={"seed": 42}
)

lista1 = [list_of_aoi[indice_aoi["RAGS 2 RICHES"]], list_of_aoi[indice_aoi["CONFLICT"]], list_of_aoi[indice_aoi["DONOR"]]]

prompt_generate_plot_schema = prompt_generate_plot_schema.replace(
    "{axis_of_interest}",
    json.dumps([a.model_dump() for a in lista1], indent=2)
)

result1 = model1.invoke(prompt_generate_plot_schema)

In [6]:
print(render_plot_schema_md(result1.content))

PLOT-SCHEMA =  RAGS 2 RICHES + DONOR + CONFLICT
DESCRIPTION:
Un héroe pasa de la pobreza a la recompensa: aspira a cambiar, es probado por un donante, obtiene y usa un agente mágico, enfrenta al villano, se transforma y vence. PROTAGONIST=Hero; ANTAGONIST=Villain; DONOR=Donor; GIFT=Gift.
────────────────────────────────────────────────────────────────
AXISofINTEREST =  RAGS 2 RICHES
PLOT-SPAN-NAME =  Rags
DESCRIPTION:
El héroe en pobreza inicial.

Poverty  characters(sufferer=Hero)
  — El héroe (sufferer=Hero) vive en la escasez.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  RAGS 2 RICHES
PLOT-SPAN-NAME =  Aspiration
DESCRIPTION:
El deseo de mejorar surge en el héroe.

Aspiration  characters(aspirer=Hero)
  — El héroe (aspirer=Hero) aspira a cambiar su destino.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  DONOR
PLOT-SPAN-NAME =  Tested
DESCRIPTION:
El donante pone a prueba al héroe y, si responde bien, promete

### Crear Cuento Completo

Dado un Plot Schema usamos un LLM para generar un cuento

In [9]:
prompt_generate_cuento = prompt_generate_cuento.replace(
    "{plot_schema}",
    result1.content
)

In [10]:
cuento = model1.invoke(prompt_generate_cuento)
print(cuento.content)

La luz en el hilo

Cuando la marea bajaba, el puerto quedaba con las costillas al aire y un olor de hierro y sal. Lía vivía en la caseta del faro apagado, una torre de piedra cansada, con ventanas selladas desde que el dueño del muelle, Fulgencio Alazán, había decretado que solo su farola privada merecía alumbrar el comercio. Lía remendaba redes a cambio de pan, contaba monedas como quien cuenta espinas, y dormía con la espalda pegada a la pared para engañar al viento. Su padre se había ido tras una tormenta; su madre había aprendido a hacerse pequeña para pasar entre las deudas; Lía heredó el faro y la pobreza.

Cada noche subía a la cúpula rota y miraba el océano negro. Soñaba con encender de nuevo la lámpara, con devolver el destello que hacía vibrar a los peces y hacía cantar a los cascos de los barcos. Soñaba con una mesa sin huecos, con tener manos que no temblaran al dar. Ese deseo, primero tímido como un vaso, acabó siendo obstinado. Lía aspiraba a cambiar, a ser la luz que aho

### Crear AoI dado un cuento

Acá lo que hacemos es el paso inverso. Queremos mandar un cuento y ver si el LLM es capaz de detectar los AoI. Hicimos dos experimentos:
- Con el cuento que creamos recien y ver si nos devuelve los AoI que usamos
- Con un cuento cualquiera, este cuento nosotros lo analizamos nosotros y detectamos algunos AoI

In [24]:
from langchain_core.prompts import PromptTemplate

prompt_devolver_aoi = PromptTemplate.from_template("Dado el siguiente cuento: {cuento}, y la siguiente lista de Axis of Interest: {axis_of_interest}, devuelve una lista con los nombres de los Axis of Interest que abarquen la historia. Tambien el orden en que se entrelazan los distintos Plot Spans.")
prompt_devolver_aoi = prompt_devolver_aoi.invoke({"cuento": cuento.content, "axis_of_interest": json.dumps([a.model_dump() for a in lista1], indent=2)})


In [25]:
aoi_devuelto = model1.invoke(prompt_devolver_aoi)

In [26]:
print(aoi_devuelto.content)

- Axis of Interest presentes en la historia:
  - RAGS 2 RICHES
  - DONOR
  - CONFLICT

- Orden (cronológico) de los Plot Spans entrelazados:
  1) RAGS 2 RICHES — Rags
  2) RAGS 2 RICHES — Aspiration
  3) DONOR — Tested
  4) CONFLICT — Struggle
  5) DONOR — UseOfAMagicalAgent
  6) RAGS 2 RICHES — Transformation
  7) CONFLICT — Victory
  8) RAGS 2 RICHES — Riches
